# Child Care Data Prep

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# read in original data
cc_df = pd.read_csv('data/original/ChildcareCenters.csv')
fc_df = pd.read_csv('data/original/FamilyChildCare.csv')

In [ ]:
# append datasets together
df = pd.concat([cc_df, fc_df], ignore_index=True, axis=0)

In [ ]:
# create flag for centers that care for under 5 (infant, toddler, pre-school)
center_service = 'infants|toddlers|preschool'
# filter out centers that don't care for under 5
df = df[df['Type Of License'].str.contains(center_service, case=False, na=False)]

In [ ]:
# filter out centers with non-active licenses
df = df[df['License Status'].str.contains('Active')]

In [ ]:
# save stage out
df.to_csv('data/modified/All_MN_Childcare.csv')

# Geocoding

In [ ]:
# strip off anything longer than 5 for zipcode
df['Zip'] = df['Zip'].astype(str).str.slice(0,5)

In [ ]:
df['Zip'].head()

In [ ]:
# concatenate address into a single string to pass to geocoder
df["full_address"] = df['AddressLine1'] + ' ' + df['City'] + ' ' + df['State'] + ' ' + df['Zip']

In [ ]:
# geocode
key = 'https://youtu.be/RfiQYRn7fBg?t=17'
locator = GoogleV3(api_key=key)

# delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=2)

# create location column
df['location'] = df['full_address'].apply(geocode)

# create longitude, laatitude from location
df['latitude'] = [g.latitude for g in df['location']]
df['longitude'] = [g.longitude for g in df['location']]

In [ ]:
# write stage out
df.to_csv('data/modified/geocoded_childcare_centers.csv')

# ACS data prep

In [89]:
import geopandas as gpd

shp_df = gpd.read_file('data/original/tl_2021_27_bg/tl_2021_27_bg.shp')
shp_df['GEOG_UNIT'] = shp_df['GEOID']
shp_df = shp_df.astype({'GEOG_UNIT':'int64'})
shp_df = shp_df.set_index(['GEOG_UNIT']).sort_index()

In [90]:
# get centroids
shp_df['coords'] = shp_df.centroid

# set datum
shp_df = shp_df.to_crs({'init': 'epsg:4326'})

shp_df['coords'].head()

C:\Users\johnd_h4rxswp\AppData\Local\Temp\ipykernel_26308\901880718.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  shp_df['coords'] = shp_df.centroid
C:\Users\johnd_h4rxswp\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


GEOG_UNIT
270017701001    POINT (-93.22335 46.87148)
270017701002    POINT (-93.42317 46.66757)
270017701003    POINT (-93.57031 46.58155)
270017702001    POINT (-93.57059 46.89993)
270017702002    POINT (-93.64448 46.69888)
Name: coords, dtype: geometry

In [91]:
# convert geopandas point to tuple
shp_df['coords'] = shp_df['coords'].apply(lambda p: tuple([p.y, p.x]))
shp_df = shp_df[['coords']]
shp_df.head()

,coords
GEOG_UNIT,
270017701001,"(46.87147737561412, -93.22334829204809)"
270017701002,"(46.66757092803079, -93.42317348039266)"
270017701003,"(46.58155228465229, -93.57031020185724)"
270017702001,"(46.89992783908026, -93.5705932582413)"
270017702002,"(46.69888457200159, -93.64448243125192)"


In [92]:
# read in ACS data and join to shp coords
acs_df = pd.read_excel('data/modified/ACS.xlsx')

acs_df = acs_df.astype({'GEOG_UNIT':'int64'})
# set index
acs_df = acs_df.set_index(['GEOG_UNIT']).sort_index()

acs_df['weight'] = acs_df['POPUNDER5'] * -1

In [93]:
acs_df.head()

,GEOG_LEVEL,GEOID,GEOID2,SUMLEV,GEONAME,GEOCOMP,STATE,COUNTY,TRACT,BLKGRP,...,MEANHHINC,MEDIANHHI,POVERTYN,POVERTY150,POV150_185,POVDENOM,POV100RATE,POV150RATE,POV185RATE,weight
GEOG_UNIT,,,,,,,,,,,,,,,,,,,,,
270017701001,BG,15000US270017701001,270017701001,150,Block Group 1 Census Tract 7701 Aitkin County ...,0,27,1,770100,1,...,69259.0,51953.0,86,63,49,846,0.102,0.176,0.234,-24
270017701002,BG,15000US270017701002,270017701002,150,Block Group 2 Census Tract 7701 Aitkin County ...,0,27,1,770100,2,...,66753.0,53625.0,131,54,56,876,0.150,0.211,0.275,-23
270017701003,BG,15000US270017701003,270017701003,150,Block Group 3 Census Tract 7701 Aitkin County ...,0,27,1,770100,3,...,67548.0,58289.0,100,76,84,683,0.146,0.258,0.381,-24
270017702001,BG,15000US270017702001,270017702001,150,Block Group 1 Census Tract 7702 Aitkin County ...,0,27,1,770200,1,...,62422.0,54643.0,96,52,98,768,0.125,0.193,0.320,-17
270017702002,BG,15000US270017702002,270017702002,150,Block Group 2 Census Tract 7702 Aitkin County ...,0,27,1,770200,2,...,63432.0,56422.0,88,62,39,742,0.119,0.202,0.255,-19


In [95]:
# join shp coords
acs_merge_df = pd.merge(acs_df, shp_df, left_index=True, right_index=True)
acs_merge_df.shape

(4706, 29)

In [96]:
# remove all extra columns
acs_merge_df = acs_merge_df[['coords', 'weight']]
acs_merge_df.shape

(4706, 2)

In [97]:
# read in child care data
cc_df = pd.read_csv('data/modified/geocoded_childcare_centers.csv')
# set index 
cc_df = cc_df.set_index(['License Number']).sort_index()

In [98]:
from shapely.geometry import Point

cc_df['coords'] = tuple(zip(cc_df['latitude'], cc_df['longitude']))
# remove the old columns
drop_cols = ['latitude', 'longitude']
cc_df = cc_df.drop(drop_cols, axis=1)

cc_df['coords'].head()

License Number
800003    (44.2917947, -93.3045992)
800020      (44.8787181, -93.38103)
800025    (45.1170803, -93.2164146)
800056    (43.6435327, -93.3647928)
800058    (44.0689429, -92.4763802)
Name: coords, dtype: object

In [99]:
# rename columns and organize
cc_df['weight'] = cc_df['Capacity']
cc_df = cc_df[['coords', 'weight']]

In [100]:
# append datasets together
final_df = pd.concat([cc_df, acs_merge_df], ignore_index=True, axis=0)
# write final data set out
final_df.to_csv('data/modified/data_to_map.csv')

In [101]:
final_df.head()

,coords,weight
0,"(44.2917947, -93.3045992)",94
1,"(44.8787181, -93.38103)",91
2,"(45.1170803, -93.2164146)",35
3,"(43.6435327, -93.3647928)",157
4,"(44.0689429, -92.4763802)",168
